In [ ]:
!pip install -U datasets bitsandbytes accelerate peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 13.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl

In [ ]:
import os
import torch
from datasets import Dataset, load_dataset
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

def set_deterministic(seed):
    # SET DETERMINISTIC
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True, warn_only=True)
    os.putenv("CUBLAS_WORKSPACE_CONFIG", ":4096:8")
    set_seed(seed, deterministic=True)

In [ ]:
dataset = load_dataset("djagatiya/synthetic_text_to_sql_d14")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6713 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'sql_complexity', 'sql_prompt', 'sql_context', 'sql'],
        num_rows: 6713
    })
    test: Dataset({
        features: ['id', 'domain', 'sql_complexity', 'sql_prompt', 'sql_context', 'sql'],
        num_rows: 408
    })
})

In [ ]:
dataset['train'][0]

{'id': 5131,
 'domain': 'healthcare',
 'sql_complexity': 'basic SQL',
 'sql_prompt': 'What is the minimum cultural competency score by worker?',
 'sql_context': 'CREATE TABLE worker_scores (worker_id INT, score INT); INSERT INTO worker_scores (worker_id, score) VALUES (1, 95), (2, 88), (3, 72);',
 'sql': 'SELECT worker_id, MIN(score) FROM worker_scores;'}

### Prepare Dataset

In [ ]:
SQL_PROMPT = """
Instruct: Write SQL query of question asked by user based on following database structure context.
Context: {context}
Question: {question}
Output: {sql}
<|endoftext|>
"""

def prepare_dataset(sample):

    sample['text'] = SQL_PROMPT.format(
        context=sample['sql_context'],
        question=sample['sql_prompt'],
        sql=sample['sql']
    ).strip()

    return sample

In [ ]:
dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/6713 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][0]['text'])

Instruct: Write SQL query of question asked by user based on following database structure context.
Context: CREATE TABLE worker_scores (worker_id INT, score INT); INSERT INTO worker_scores (worker_id, score) VALUES (1, 95), (2, 88), (3, 72);
Question: What is the minimum cultural competency score by worker?
Output: SELECT worker_id, MIN(score) FROM worker_scores;
<|endoftext|>


### Model Building

In [ ]:
model_name='microsoft/phi-2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    use_fast=False,
    trust_remote_code=True
)
tokenizer.add_special_tokens({'pad_token': '<|padding|>'})
print(tokenizer.all_special_tokens)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['<|endoftext|>', '<|padding|>']


In [ ]:
encoded = tokenizer("Hello world<|endoftext|>", add_special_tokens=True, max_length=10, padding='max_length')
encoded

{'input_ids': [50295, 50295, 50295, 50295, 50295, 50295, 50295, 15496, 995, 50256], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(encoded['input_ids'], skip_special_tokens=False)

['<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 '<|padding|>',
 'Hello',
 'Ġworld',
 '<|endoftext|>']

In [ ]:
train_max_len = max([len(tokenizer.encode(t)) for t in dataset['train']['text']])
train_max_len

534

In [ ]:
# Free memory
import gc
if 'model' in globals():
    print("Existing model deleted.")
    del model
gc.collect()
torch.cuda.empty_cache()

# Set Seed
set_deterministic(1)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    quantization_config=bnb_config,
)

model.resize_token_embeddings(len(tokenizer))

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "mlp.fc1", "mlp.fc2"],
    modules_to_save=["lm_head", "embed_tokens"]
)

model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

Existing model deleted.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 330,966,136 || all params: 3,106,020,592 || trainable%: 10.6556


In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import DataCollatorForLanguageModeling, TrainingArguments

In [ ]:
response_template_with_context = "\nOutput:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)
response_template_ids

[198, 26410, 25]

In [ ]:
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
train_ds, test_ds = dataset['train'], dataset['test']

In [ ]:
args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    # learning_rate=1e-4,
    optim="adamw_8bit",
    output_dir="model2",
    # save_embedding_layers=True,
    logging_strategy='steps',
    save_strategy='steps',
    evaluation_strategy='steps',
    eval_steps=300,
    logging_steps=300,
    save_steps=300,
    save_only_model=True
)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    data_collator=collator,
    args=args,
    dataset_text_field='text',
    train_dataset=train_ds,
    eval_dataset=test_ds,
    max_seq_length=600
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss,Validation Loss
300,0.197300,0.170346
600,0.189900,0.162319
900,0.177800,0.159404
1200,0.176000,0.154415


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Step,Training Loss,Validation Loss
300,0.197300,0.170346
600,0.189900,0.162319
900,0.177800,0.159404
1200,0.176000,0.154415
1500,0.160100,0.149537
1800,0.147500,0.145144
2100,0.138400,0.139313
2400,0.133900,0.135821
2700,0.131400,0.136104
3000,0.125800,0.135440


TrainOutput(global_step=3356, training_loss=0.15461336415487478, metrics={'train_runtime': 7930.59, 'train_samples_per_second': 1.693, 'train_steps_per_second': 0.423, 'total_flos': 4.043936056164211e+16, 'train_loss': 0.15461336415487478, 'epoch': 1.9997020706092656})

### Evaluation

In [ ]:
!zip -rj checkpoint-3300.zip /content/model2/checkpoint-3300

  adding: trainer_state.json (deflated 75%)
  adding: tokenizer_config.json (deflated 93%)
  adding: README.md (deflated 66%)
  adding: special_tokens_map.json (deflated 79%)
  adding: added_tokens.json (deflated 82%)
  adding: merges.txt (deflated 53%)
  adding: adapter_config.json (deflated 52%)
  adding: vocab.json (deflated 68%)
  adding: training_args.bin (deflated 51%)
  adding: adapter_model.safetensors (deflated 8%)


In [ ]:
!cp /content/checkpoint-3300.zip /content/drive/MyDrive

In [ ]:
!unzip /content/drive/MyDrive/checkpoint-3300.zip -d phi2_text2sql

Archive:  /content/drive/MyDrive/checkpoint-3300.zip
  inflating: phi2_text2sql/trainer_state.json  
  inflating: phi2_text2sql/tokenizer_config.json  
  inflating: phi2_text2sql/README.md  
  inflating: phi2_text2sql/special_tokens_map.json  
  inflating: phi2_text2sql/added_tokens.json  
  inflating: phi2_text2sql/merges.txt  
  inflating: phi2_text2sql/adapter_config.json  
  inflating: phi2_text2sql/vocab.json  
  inflating: phi2_text2sql/training_args.bin  
  inflating: phi2_text2sql/adapter_model.safetensors  


In [ ]:
load_model_name = '/content/phi2_text2sql'

tokenizer = AutoTokenizer.from_pretrained(
    load_model_name,
    padding_side="left",
    use_fast=False,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    'microsoft/phi-2',
    device_map=0,
    quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False)
)

model.resize_token_embeddings(len(tokenizer))

from peft import PeftModel

# Load the LoRA model
inference_model = PeftModel.from_pretrained(model, load_model_name)
inference_model = inference_model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inference_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(50296, 2560)
          (modules_to_save): ModuleDict(
            (default): Embedding(50296, 2560)
          )
        )
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2560, bias=False)
           

In [ ]:
import sqlite3

def query_execution_eval(context, true_sql, pred_sql):
    try:
        db = sqlite3.connect(":memory:")
        cur = db.cursor()
        cur.executescript(context)
        true_result = str(cur.execute(true_sql).fetchall())
        pred_result = str(cur.execute(pred_sql).fetchall())
        return true_result == pred_result
    except:
        return False

In [ ]:
PRED_SQL_PROMPT = """
Instruct: Write SQL query of question asked by user based on following database structure context.
Context: {context}
Question: {question}
Output:"""

def prediction(samples):

    inputs = []
    for i in range(len(samples['id'])):
        inputs.append(PRED_SQL_PROMPT.format(
            context=samples['sql_context'][i],
            question=samples['sql_prompt'][i],
        ))

    # print(inputs)

    model_inputs = tokenizer(inputs, padding=True, return_tensors="pt").to("cuda")
    # print(model_inputs)

    generated_ids = inference_model.generate(**model_inputs, max_new_tokens=500)
    result = tokenizer.batch_decode(generated_ids[:, model_inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    result = [r.strip() for r in result]

    # AVOID REPEATATIVE
    for i in range(len(result)):
        r = result[i]
        stop_index = r.find(";")
        if stop_index != -1:
            result[i] = r[:stop_index+1]

    return result

In [ ]:
import math

start_index, batch_size = 0, 8

all_evals = []
pred_sqls = []

for i in range(math.ceil(len(test_ds) / batch_size)):

    end_index = start_index + batch_size

    print(start_index, end_index)
    samples = test_ds[start_index:end_index]
    results = prediction(samples)
    print(results)
    pred_sqls.extend(results)

    eval_result = [query_execution_eval(context, sql, pred) \
        for context, sql, pred in \
        zip(samples['sql_context'], samples['sql'], results)]

    print(eval_result)

    all_evals.extend(eval_result)

    start_index = end_index

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 8


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["SELECT name FROM renewable_project WHERE country = 'India' AND budget BETWEEN 50.0 AND 200.0;", "SELECT AVG(energy_rating) FROM energy_efficiency WHERE building_type = 'Commercial' AND location = 'Texas';", 'SELECT player, AVG(running_speed) FROM world_cup WHERE match_id = 2020 GROUP BY player;', 'SELECT name, total_transactions FROM shariah_compliant_institutions;', 'SELECT gender, race, SUM(reoffender) as reoffenders_2017, SUM(reoffenders_2017) as total_reoffenders_2017, SUM(reoffenders_2018) as reoffenders_2018, SUM(reoffenders_2018) as total_reoffenders_2018 FROM parolee WHERE year IN (2017, 2018) GROUP BY gender, race;', 'SELECT district_id, SUM(cases) FROM restorative_justice GROUP BY district_id;', 'SELECT Director, AVG(Rating) as AvgRating FROM DirectorMoviesRating GROUP BY Director;', "SELECT AVG(rating) FROM movies WHERE production_year BETWEEN 2010 AND 2020 AND country = 'USA';"]
[True, False, False, False, False, True, True, True]
8 16


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

["SELECT COUNT(*) FROM digital_divide_initiatives WHERE region = 'Asia';", "SELECT name FROM farms WHERE size > 150 AND location = 'Texas';", "SELECT SUM(revenue) FROM sales_view WHERE farmer_id = 1 AND crop_name = 'Potatoes';", "SELECT ArtistName, SUM(SalesAmount) as TotalRevenue FROM MusicSales WHERE Genre = 'Digital' GROUP BY ArtistName ORDER BY TotalRevenue DESC LIMIT 3;", "SELECT AVG(Budget) FROM Accessible_Tech_Projects WHERE Location = 'Africa';", 'SELECT name, MAX(balance) FROM shariah_compliant_finance GROUP BY name;', 'SELECT WarehouseManagementTransactions.TransactionID, WarehouseManagementTransactions.TransactionStatus, WarehouseManagementTransactions.TransactionDate FROM WarehouseManagementTransactions WHERE WarehouseManagementTransactions.WarehouseID = 3;', "SELECT COUNT(*) FROM community_health_workers WHERE community IN ('First Nations', 'Inuit', 'Métis');"]
[True, True, True, False, True, False, True, True]
384 392


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["SELECT MAX(altitude) FROM RocketAltitudes WHERE rocket = 'Falcon 9';", 'SELECT country, SUM(cost) FROM satellites GROUP BY country;', 'SELECT Country, AVG(Consumption) as AvgConsumptionPerCapita FROM EnergyConsumptionPerCapita WHERE Year = 2021 GROUP BY Country;', "SELECT MAX(usage) FROM power_usage WHERE building_type = 'Industrial' AND location = 'California';", "SELECT matches FROM tennis_tournaments WHERE court = 'Grass';", "SELECT SUM(fans_attended) FROM matches WHERE team = 'Manchester United' AND year = 2020;", 'SELECT Genre, SUM(RunningTime) FROM GenreRunningTimes GROUP BY Genre;', "SELECT products.name, suppliers.name FROM products INNER JOIN suppliers ON products.supplier_id = suppliers.id WHERE suppliers.name!= 'Green Cotton Inc.';"]
[True, True, True, False, False, True, True, True]
392 400


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["SELECT SUM(mass) FROM gso_debris WHERE orbit = 'GSO' AND source = 'Defunct Satellite';", 'SELECT MAX(cost) FROM rover_missions;', 'SELECT SUM(cost) FROM space_missions;', 'SELECT s.name, AVG(l.progress) as avg_progress FROM lifelong_learning l JOIN schools s ON l.school_id = s.school_id GROUP BY s.name;', "SELECT COUNT(cards) FROM yellow_cards JOIN teams ON yellow_cards.team_id = teams.team_id WHERE teams.name = 'Bayern Munich' AND tournament = 'Champions League';", 'SELECT COUNT(*) FROM clients JOIN loans ON clients.client_id = loans.client_id WHERE clients.is_financially_capable = true;', "SELECT COUNT(*) FROM Warehouses WHERE city = 'City Y' AND capacity > 100000;", "SELECT outlet_name, COUNT(*) as num_articles FROM media_outlets JOIN fact_checks ON media_outlets.outlet_id = fact_checks.outlet_id WHERE fact_checks.is_true = TRUE AND fact_checks.fact_check_date >= '2021-01-01' GROUP BY outlet_name ORDER BY num_articles DESC LIMIT 5;"]
[True, False, True, False, False, True, True, T

In [ ]:
len(all_evals), np.array(all_evals).mean()

(408, 0.6936274509803921)

In [ ]:
# test_ds = test_ds.remove_columns(['score', 'pred_sql'])
test_ds = test_ds.add_column('score', np.array(all_evals))
test_ds = test_ds.add_column('pred_sql', pred_sqls)

In [ ]:
check_df = pd.DataFrame(test_ds)

In [ ]:
check_df.groupby(['sql_complexity'])['score'].mean()

sql_complexity
aggregation    0.631148
basic SQL      0.756637
single join    0.583333
Name: score, dtype: float64